Munges CSV to JSON for playlist graph.

In [289]:
import json
import random
import lyricsgenius # has aggressive versioning requirements
import spotipy

In [290]:
rows = []
with open('thumbprint.csv') as f:
    cols = [s.strip() for s in f.readline().split(',')]
    for line in f:
        if '---' not in line:
            # linker will remove duplicates
            rows.append( dict(zip(cols, [s.strip() for s in line.split(',')])) )

        # else: rows.append( dict(empty=True) )
rows[-2]

{'artist': 'maniac bloom', 'song': 'everything i saved you from'}

In [291]:
links = []

u = rows[0]
nodes = [u]

i,j = 0,1 # indexes of u,v
k = 0 # duplicate count

for v in rows[1:]:
    if v in nodes:
        j = nodes.index(v)
        k += 1
    else:
        j = len(nodes)
        nodes.append(v)
    
    if i != 0 and j != 0:
        links.append( dict(source=i, target=j) )
    i = j
    u = v
    
k

45

#### Annotate tracks.
WIP

In [292]:
u = nodes[random.randint(0, len(nodes))]

with open('token.txt', 'r') as f:
    genius_token = f.readline().strip()
    spotify_id = f.readline().strip()
    spotify_token = f.readline().strip()

In [293]:
api = lyricsgenius.Genius(genius_token)
res = api.search_song(u['song'], u['artist'])
res

Searching for "the twyncyn / trembling willows" by pure reason revolution...
Done.


('The Twyncyn / Trembling Willows', 'Pure Reason Revolution')

In [297]:
verses = res.lyrics.split('\n')
verses = [s for s in verses if '[' not in s and s is not '']

r = 3
i = random.randint(0,len(verses)-r)

print( verses[i:i+r] )

['Through warm shine', 'Clear as the fountains', 'Round the twyncyn']


In [298]:
from spotipy.oauth2 import SpotifyClientCredentials

man = SpotifyClientCredentials(client_id=spotify_id,
                               client_secret=spotify_token)
sp = spotipy.Spotify(client_credentials_manager=man)

In [302]:
res = sp.search(q='track:' + u['song'] + ' artist:' + u['artist'],
                type='track', limit=1)
v = res['tracks']['items']
v[0].keys()

IndexError: list index out of range

#### Annotate tracks with duration.

In [303]:
for i,u in enumerate(nodes):
    res = sp.search(q='track:' + u['song'] + ' artist:' + u['artist'],
                        type='track', limit=1)

    v = res['tracks']['items']
    if len(v) > 0:
        u['duration'] = v[0]['duration_ms'] / 1000
        u['year'] = v[0]['album']['release_date'].split('-')[0]
        u['weight'] = v[0]['popularity']
        
    if i%50 == 0:
        print('...')

    # seconds = v['duration_ms'] / 1000
    # m,s = int(seconds//60), int(seconds%60)

...
...
...


In [281]:
g = links[random.randint(0, len(links))]
# for g in links:
i,j = g['source'], g['target']
print(nodes[i], nodes[j])

res = []
for u in [nodes[i], nodes[j]]:
    res.append( api.search_song(u['song'], u['artist']) )
    
lhs = [s for s in res[0].lyrics.split('\n') if '[' not in s and s is not '']
rhs = [s for s in res[1].lyrics.split('\n') if '[' not in s and s is not '']

{'song': 'dancer in the dark', 'artist': 'the rasmus'} {'song': 'paint it black', 'artist': 'ciara', 'duration': 234.146, 'year': '2015', 'weight': 57}
Searching for "dancer in the dark" by the rasmus...
Done.
Searching for "paint it black" by ciara...
Done.


In [287]:
# g['lhs'] = random.choice(lhs)
# g['rhs'] = random.choice(rhs)
# g['lhs'], g['rhs']

('You heard it calling, but you turned your face, never played your part',
 'My love will laugh with me before the morning comes')

FIXME: Cache lyrics? Project onto the actual links, not UI space. Vary transmissions in realtime.

#### Write to json.

In [304]:
for i,u in enumerate(nodes):
    u['id'] = i

with open("thumbprint-graph.json", "w") as f:
    f.write( json.dumps(dict(nodes=nodes[1:], links=links)) )

Sadly, the 'a-scene' likes to take over the entire notebook, instead of being a widget. 